In [4]:
import tkinter as tk
from tkinter import *
      #------commentaires------:
# dans cette class on a : 
# la creation du canon
#le deplacement du defender
# fonction fire pour controler le nombre autorise de bullet
# fonction move_bullet pour controler le deplacement du bullet 
class Defender(object):
    def __init__(self): 
        self.width = 20
        self.height = 20
        self.move_delta = 15 
        self.id = None
        self.max_fired_bullets = 8 
        self.fired_bullets = []
        self.canvas_height = Fleet().get_height()
        self.canvas_width = Fleet().get_width()
        self.dx = self.canvas_width//2 - self.width//2
        self.dy = self.canvas_height - self.height        
    def get_id(self):
        return self.id
    def get_fire(self):
        return self.fired_bullets
    def set_fire(self,liste):
        self.fired_bullets=liste
    def get_dx(self):
        return self.dx
    def get_dy(self):
        return self.dy
    def set_dx(self,valeurx):
        self.dx=valeurx
    def set_dy(self,valeury):
        self.dy=valeury
        
    def install_in(self, canvas):
        self.canvas=canvas
        lx = self.get_dx()
        ly = self.get_dy()
        self.rectangle = canvas.create_rectangle(lx, ly, lx + self.width, ly + self.height, fill="white")
        
    def keypress(self, event):
        x = 0
        xn=self.get_dx()  
        if event.keysym == 'Left':
            if xn > 0:   
                self.set_dx(self.get_dx()-self.move_delta)    
                self.move_in(self.canvas,-self.move_delta)     
        elif event.keysym == 'Right':
            if xn+self.move_delta < self.canvas_width:   
                self.set_dx(self.get_dx()+self.move_delta)   
                self.move_in(self.canvas,self.move_delta)     
        elif event.keysym == 'space':  
            self.id = 1
            self.bullet = Bullet(self.id)
            self.bullet.set_xb(self.get_dx())    
            self.bullet.set_yb(self.get_dy())
            self.fire(self.canvas)
        
    def move_in(self,canvas, dx): 
        canvas.move(self.rectangle, dx, 0)
       
    def fire(self, canvas):
        if len(self.get_fire()) < self.max_fired_bullets:     
            self.bullet.id = self.bullet.install_in(self.canvas)    
            self.fired_bullets.append(self.bullet.id) 
    
    def move_bullet(self,canvas):
        for i in range(0,len(self.fired_bullets)):
            x1,y1,x2,y2 = self.canvas.bbox(self.fired_bullets[i])
            if y1<0:    
                canvas.delete(self.fired_bullets[i])    
                del self.fired_bullets[i]   
                break
            else:
                self.bullet.move_in(self.canvas,self.fired_bullets[i])  
                
        #------commentaires------:
#creation du bullet 
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 10
        self.color = "red"
        self.speed = 8
        self.id = None
        self.shooter = shooter
        self.x=Defender().get_dx()
        self.y=Defender().get_dy()
    def get_id(self):
        return self.id
    def set_id(self,bid): 
        self.id = bid
    def get_xb(self):      
        return self.xb
    def get_yb(self):
        return self.yb
    def set_xb(self,valeurx):
        self.xb=valeurx
    def set_yb(self,valeury):
        self.yb=valeury
        
    def install_in(self, canvas):
        lx = self.get_xb() + 5
        ly = self.get_yb() - 10
        self.id = canvas.create_oval(lx, ly, lx + self.radius, ly + self.radius, fill="red")
        return self.id
    
    def move_in(self,canvas,ovale):
        self.id=ovale
        canvas.move(self.id, 0, -self.speed)

        
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 4
        self.aliens_columns = 6
        self.aliens_inner_gap = 20
        self.alien_x_delta = 5
        self.alien_y_delta = 15
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size
        self.width = 700
        self.height =600
    
    def get_fx(self):
        return self.alien_x_delta
    def get_fy(self):
        return self.alien_y_delta
    def set_fx(self,fx):
        self.alien_x_delta=fx
    def get_width(self):
        return self.width
    def get_height(self):
        return self.height
        
    def install_in(self, canvas):
        self.alien=Alien()
        self.canvas=canvas
        self.x=50  
        self.y=50
        position=0 
        
        for i in range(0,self.aliens_lines):
            for j in range(0,self.aliens_columns):
                self.aliens_fleet[position] = self.alien.install_in(self.canvas,self.x,self.y)
                position+=1
                self.x += self.aliens_inner_gap+73  
            self.x=50
            self.y += self.aliens_inner_gap+53  
           
    def move_in(self, canvas): 
        if len(self.aliens_fleet)!=0:
            all_rect_ids = self.canvas.find_withtag("alien")
            x1,y1,x2,y2 = self.canvas.bbox("alien")
            
            if x2>=788:      
                self.set_fx(-self.get_fx())
                fy = self.alien_y_delta
            elif x1<=0:      
                self.set_fx(-self.get_fx())
                fy = self.alien_y_delta
            else:       
                fy=0

            for i in range(0,len(all_rect_ids)):
                self.alien.move_in(self.canvas,all_rect_ids[i],self.alien_x_delta,fy)
                
     #---------commentaires-----:
# dans fonction on va faire le conditions suivantes:
# si des alients vivants existent
# est ce que le bullet a toucher un alien 
# ensuite on va suprimer le bullet et l'alien 

    def fonction(self,canvas,defender):
        self.canvas=canvas
        self.defender=defender
        
        for i in range(len(self.defender.fired_bullets)):
            cond_sorti=0   
#coordonnes des bullets
            xb1,yb1,xb2,yb2=self.canvas.bbox(self.defender.fired_bullets[i]) 
            for j in range(len(self.aliens_fleet)):
    
                if self.aliens_fleet[j] != None: 
                    xa1,ya1,xa2,ya2=self.canvas.bbox(self.aliens_fleet[j])

                    if (xb1>=xa1 or xb2>=xa1) and (xb1<=xa2 or xb2<=xa2) and yb1<=ya2 and yb1>=ya1:
                        self.alien.expo(self.canvas,self.defender.fired_bullets[i]) 
                        canvas.delete(self.defender.fired_bullets[i])  
                        canvas.delete(self.aliens_fleet[j])    
                        self.alien.alive = False
      
                        del self.defender.fired_bullets[i]   
                        del self.aliens_fleet[j] 
                        cond_sorti=1    
                        break
                        
            if(cond_sorti==1):
                break     
                
        #-----------commentaires----------:                
 #dans cette class on a :
 # la creation d'un alien 
 # le deplacement de l'alien 
 # creer l'explosion
class Alien(object):
    def __init__(self):
        self.id = None
        self.alive = True
        self.alien = PhotoImage(file="alien.gif")
        self.explosion = PhotoImage(file="explosion.gif")
 
    def install_in(self, canvas, x, y):  
        self.id = canvas.create_image(x, y, image=self.alien, tags="alien")
        return self.id
    
    def move_in(self, canvas, alien, dx, dy):
        self.id=alien
        canvas.move(self.id, dx, dy)

    def expo(self,canvas,projectile):
        xa1,ya1,xa2,ya2=canvas.bbox(projectile)
        explosion = canvas.create_image(xa1+(xa2-xa1)/2, ya1+(ya2-ya1)/2, image=self.explosion, tags="explosion")
        canvas.after(100,canvas.delete,explosion)        
        

class Game(object):
    
    def __init__(self, frame):
        width=700
        height=600
        self.frame=frame
        self.canvas=tk.Canvas(self.frame,width=width, height=height,bg="black")
        self.canvas.pack(side="top", fill="both", expand=True)
        self.fleet=Fleet()
        self.defender=Defender()
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
       
    def start_animation(self):
        self.animation()

    def animation(self):
        self.move_bullets()
        self.move_aliens_fleet()
        self.fleet.fonction(self.canvas,self.defender)
        self.canvas.after(100,self.animation)

    def move_bullets(self):
        if self.defender.get_id() == 1:
            self.defender.move_bullet(self.canvas)
            
    def move_aliens_fleet(self):
        self.fleet.move_in(self.canvas)          
           
                        
class SpaceInvaders(object): 
    ''' Main Game class '''
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        width=700
        height=600
        self.frame=tk.Frame(self.root,width=width, height=height,bg="green")
        self.frame.pack()
        self.game = Game(self.frame)
        
    def play(self): 
        self.game.start_animation()
        self.root.bind("<Key>", self.game.defender.keypress)
        self.root.mainloop()        
                
jeu=SpaceInvaders()
jeu.play()

